<a href="https://colab.research.google.com/github/arpitsharma27/PythonPractice/blob/main/UniversityOf_Michigan_InferentialStatistical_Week2_LabExercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Statistical Inference with Confidence Intervals

Throughout week 2, we have explored the concept of confidence intervals, how to calculate them, interpret them, and what confidence really means.  

In this tutorial, we're going to review how to calculate confidence intervals of population proportions and means.

To begin, let's go over some of the material from this week and why confidence intervals are useful tools when deriving insights from data.

### Why Confidence Intervals?

Confidence intervals are a calculated range or boundary around a parameter or a statistic that is supported mathematically with a certain level of confidence.  For example, in the lecture, we estimated, with 95% confidence, that the population proportion of parents with a toddler that use a car seat for all travel with their toddler was somewhere between 82.2% and 87.7%.

This is *__different__* than having a 95% probability that the true population proportion is within our confidence interval.

Essentially, if we were to repeat this process, 95% of our calculated confidence intervals would contain the true proportion.

### How are Confidence Intervals Calculated?

Our equation for calculating confidence intervals is as follows:

$$Best\ Estimate \pm Margin\ of\ Error$$

Where the *Best Estimate* is the **observed population proportion or mean** and the *Margin of Error* is the **t-multiplier**.

The t-multiplier is calculated based on the degrees of freedom and desired confidence level.  For samples with more than 30 observations and a confidence level of 95%, the t-multiplier is 1.96

The equation to create a 95% confidence interval can also be shown as:

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

Lastly, the Standard Error is calculated differenly for population proportion and mean:

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}}$$

Let's replicate the car seat example from lecture:

In [1]:
import numpy as np

In [3]:
tstar=1.96
p=0.85
n=659

se=np.sqrt((p*(1-p))/n)
se

0.01390952774409444

In [5]:
lcb=p-tstar *se
ucb=p+tstar*se
(lcb,ucb)

(0.8227373256215749, 0.8772626743784251)

In [6]:
import statsmodels.api as sm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [7]:
sm.stats.proportion_confint(n*p,n)

(0.8227378265796143, 0.8772621734203857)

In [9]:
import pandas as pd

url="https://raw.githubusercontent.com/arpitsharma27/PythonPractice/main/Coursera_Cartwheeldata.csv"
df=pd.read_csv(url)

In [10]:
df.head()

,ID,Age,Gender,GenderGroup,Glasses,GlassesGroup,Height,Wingspan,CWDistance,Complete,CompleteGroup,Score
0,1,56,F,1,Y,1,62.0,61.0,79,Y,1,7
1,2,26,F,1,Y,1,62.0,60.0,70,Y,1,8
2,3,33,F,1,Y,1,66.0,64.0,85,Y,1,7
3,4,39,F,1,N,0,64.0,63.0,87,Y,1,10
4,5,27,M,2,N,0,73.0,75.0,72,N,0,4


In [13]:
mean=df["CWDistance"].mean()
sd=df["CWDistance"].std()
n=len(df)

print(mean)
print(sd)
print(n)

82.48
15.058552387264855
25


In [14]:
tstar=2.064
se=sd/np.sqrt(n)
se

3.0117104774529713

In [15]:
lcb= mean-tstar*se
ucb=mean+tstar*se
(lcb,ucb)

(76.26382957453707, 88.69617042546294)

In [16]:
sm.stats.DescrStatsW(df["CWDistance"]).zconfint_mean()

(76.57715593233024, 88.38284406766977)

# Confidence Intervals


This tutorial is going to demonstrate how to load data, clean/manipulate a dataset, and construct a confidence interval for the difference between two population proportions and means.

We will use the 2015-2016 wave of the NHANES data for our analysis.

*Note: We have provided a notebook that includes more analysis, with examples of confidence intervals for one population proportions and means, in addition to the analysis I will show you in this tutorial.  I highly recommend checking it out!

For our population proportions, we will analyze the difference of proportion between female and male smokers.  The column that specifies smoker and non-smoker is "SMQ020" in our dataset.

For our population means, we will analyze the difference of mean of body mass index within our female and male populations.  The column that includes the body mass index value is "BMXBMI".

Additionally, the gender is specified in the column "RIAGENDR".

In [19]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [20]:
url="https://raw.githubusercontent.com/arpitsharma27/PythonPractice/main/Coursera_nhanes_2015_2016.csv"
da=pd.read_csv(url)

## **Investigating and Cleaning Data**

In [23]:
#Recode SMQ020 from 1/2 to Yes/No into new variable SMQ020x
da["SMQ020x"]=da.SMQ020.replace({1:'Yes', 2:'No', 7:np.nan, 9:np.nan})
da["SMQ020x"]

0       Yes
1       Yes
2       Yes
3        No
4        No
       ... 
5730    Yes
5731     No
5732    Yes
5733    Yes
5734     No
Name: SMQ020x, Length: 5735, dtype: object

In [24]:
da.SMQ020x.value_counts()

No     3406
Yes    2319
Name: SMQ020x, dtype: int64

In [26]:
#Recode RIAGENDR from 1/2 to Male/Female into new variable RIAGENDRx
da["RIAGENDRx"]=da.RIAGENDR.replace({1:'Male',2:'Female'})
da["RIAGENDRx"]

0         Male
1         Male
2         Male
3       Female
4       Female
         ...  
5730    Female
5731      Male
5732    Female
5733      Male
5734    Female
Name: RIAGENDRx, Length: 5735, dtype: object

In [27]:
da.RIAGENDRx.value_counts()

Female    2976
Male      2759
Name: RIAGENDRx, dtype: int64

In [28]:
dx=da[["SMQ020x","RIAGENDRx"]].dropna()
pd.crosstab(dx.SMQ020x,dx.RIAGENDRx)

RIAGENDRx,Female,Male
SMQ020x,,
No,2066,1340
Yes,906,1413


In [29]:
#Recode SMQ020x from Yes/No to 1/0 into existing variable SMQ020x
dx["SMQ020x"] = dx.SMQ020x.replace({"Yes":1, "No":2})

In [30]:
dz=dx.groupby("RIAGENDRx").agg({"SMQ020x": [np.mean, np.size]})
dz.columns = ["Proportion", "Total n"]
dz

,Proportion,Total n
RIAGENDRx,,
Female,1.695155,2972
Male,1.486742,2753


### Constructing Confidence Intervals

Now that we have the population proportions of male and female smokers, we can begin to calculate confidence intervals.  From lecture, we know that the equation is as follows:

$$Best\ Estimate \pm Margin\ of\ Error$$

Where the *Best Estimate* is the **observed population proportion or mean** from the sample and the *Margin of Error* is the **t-multiplier**.

The equation to create a 95% confidence interval can also be shown as:

$$Population\ Proportion\ or\ Mean\ \pm (t-multiplier *\ Standard\ Error)$$

The Standard Error (SE) is calculated differenly for population proportion and mean:

$$Standard\ Error \ for\ Population\ Proportion = \sqrt{\frac{Population\ Proportion * (1 - Population\ Proportion)}{Number\ Of\ Observations}}$$

$$Standard\ Error \ for\ Mean = \frac{Standard\ Deviation}{\sqrt{Number\ Of\ Observations}}$$

Lastly, the standard error for difference of population proportions and means is:

$$Standard\ Error\ for\ Difference\ of\ Two\ Population\ Proportions\ Or\ Means = \sqrt{(SE_{\ 1})^2 + (SE_{\ 2})^2}$$

#### Difference of Two Population Proportions

In [31]:
p=0.304845
n = 2972
se_female = np.sqrt(p*(1-p)/n)
se_female

0.00844415041930423

In [32]:
p=0.513258
n=2753
se_male=np.sqrt(p*(1-p)/n)
se_male

0.009526078787008965

In [33]:
se_diff=np.sqrt((se_female)**2+(se_male)**2)
se_diff

0.012729880335656654

In [34]:
d=0.304845-0.513258
lcb=d-1.96*se_diff
ucb=d+1.96*se_diff
(lcb, ucb)

(-0.23336356545788706, -0.18346243454211297)

**Difference of Two Population Means**

In [35]:
da["BMXBMI"].head()

0    27.8
1    30.8
2    28.8
3    42.4
4    20.3
Name: BMXBMI, dtype: float64

In [36]:
da.groupby("RIAGENDRx").agg({"BMXBMI": [np.mean, np.std, np.size]})

BMXBMI                  
                mean       std    size
RIAGENDRx                             
Female     29.939946  7.753319  2976.0
Male       28.778072  6.252568  2759.0

In [37]:
sem_female=7.753319/np.sqrt(2976)
sem_male = 6.252568/np.sqrt(2759)
(sem_female, sem_male)

(0.14212523289878048, 0.11903716451870151)

In [38]:
sem_diff=np.sqrt(sem_female**2+sem_male**2)
sem_diff

0.18538993598139303

In [39]:
d=29.939946 - 28.778072

In [41]:
lcb=d-1.96 * sem_diff
ucb=d+1.96 * sem_diff
(lcb,ucb)

(0.798509725476467, 1.5252382745235278)